### NCBI のチョウザメゲノムを探ってみる

NCBI にはどのようなチョウザメ (acipenser) のゲノム情報が登録されているのか調べてみます。

同時に [NCBI Datasets](https://ncbiinsights.ncbi.nlm.nih.gov/2022/10/12/ncbi-datasets-command-line-tools/) の `datasets` の挙動を調べていきます。JSONデータの加工には `jq` を使います。

まずは `datasets --help` を頼りに次のコマンドを入れてみます。

In [1]:
datasets summary genome taxon 'acipenser'

{"reports": [{"accession":"GCA_024762115.1","assembly_info":{"assembly_level":"Contig","assembly_name":"AG_genome","assembly_status":"current","assembly_type":"haploid","bioproject_lineage":[{"bioprojects":[{"accession":"PRJNA851573","title":"Acipenser gueldenstaedtii Genome sequencing and assembly"}]}],"biosample":{"accession":"SAMN29232907","attributes":[{"name":"isolate","value":"M9"},{"name":"collection_date","value":"2018"},{"name":"depth","value":"0"},{"name":"env_broad_scale","value":"Artificial pool"},{"name":"env_local_scale","value":"Dan Fish Farm"},{"name":"env_medium","value":"Artificial pool"},{"name":"geo_loc_name","value":"Israel: Upper Galilee, Dan Fish Farm"},{"name":"isol_growth_condt","value":"https://doi.org/10.1111/age.13188"},{"name":"lat_lon","value":"33.2355 N 35.6570 E"},{"name":"sex","value":"male"},{"name":"age","value":"4 years old"}],"bioprojects":[{"accession":"PRJNA851573"}],"description":{"comment":"Keywords: GSC:MIxS;MIGS:5.0","organism":{"organism_name

早速大量のデータが並んで訳が分かりません。`jq` を使って keys だけを見てみます。

In [2]:
datasets summary genome taxon 'acipenser' | 
  jq 'keys'

[
  "reports",
  "total_count"
]


これでトップレベルには2つのキーしかないことが分かりました。`total_count` キーの値を出してみます。

In [3]:
datasets summary genome taxon 'acipenser' | 
  jq '.total_count'

10


また `reports` の種類 も見てみます。

In [4]:
datasets summary genome taxon 'acipenser' | 
  jq '.reports | type'

"array"


…以上より、`reports` は値としてリストを持っていて `total_counts` にその要素数が入っていることが判明しました。

では `reports` リストの先頭要素のキーを見てみます。

In [5]:
datasets summary genome taxon 'acipenser' | 
  jq '.reports[0] | keys'

[
  "accession",
  "assembly_info",
  "assembly_stats",
  "current_accession",
  "organism",
  "source_database",
  "wgs_info"
]


更に深掘りして organism の部分を見てみます。

In [6]:
datasets summary genome taxon 'acipenser' | 
  jq '.reports[0] | .organism'

{
  "common_name": "Russian sturgeon",
  "infraspecific_names": {
    "isolate": "M9",
    "sex": "male"
  },
  "organism_name": "Acipenser gueldenstaedtii",
  "tax_id": 7902
}


これでだいたいの構造が分かりました。

そこで `reports` の全エントリから `tax_id`, `common_name`, `organism_name`, `accession` を抽出してみます。

In [7]:
datasets summary genome taxon 'acipenser' | 
  jq '.reports[] | [ .organism.tax_id, .organism.common_name, .organism.organism_name, .accession ]'

[
  7902,
  "Russian sturgeon",
  "Acipenser gueldenstaedtii",
  "GCA_024762115.1"
]
[
  40147,
  null,
  "Acipenser oxyrinchus oxyrinchus",
  "GCA_013184475.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCF_010645085.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCF_902713435.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCA_004119895.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCA_010645085.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCA_902713425.2"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCA_902713435.1"
]
[
  7906,
  "sterlet",
  "Acipenser ruthenus",
  "GCA_902713435.2"
]
[
  7904,
  "white sturgeon",
  "Acipenser transmontanus",
  "GCA_020113195.1"
]


まだ長くて見づらいので表形式にしてみます。

ここで `sed 's/\\"//g'` は `\"` の除去、`sed 's/"//g'` は `"` の除去です。

`xsv table` は表形式に加工してくれます。

In [1]:
 datasets summary genome taxon 'acipenser' | 
   jq '.reports[] | [ .organism.tax_id, .organism.common_name, .organism.organism_name, .accession ] | @csv' | 
   sed 's/\\"//g' | 
   sed 's/"//g' | 
   xsv table

7902   Russian sturgeon  Acipenser gueldenstaedtii        GCA_024762115.1
40147                    Acipenser oxyrinchus oxyrinchus  GCA_013184475.1
7906   sterlet           Acipenser ruthenus               GCF_010645085.1
7906   sterlet           Acipenser ruthenus               GCF_902713435.1
7906   sterlet           Acipenser ruthenus               GCA_004119895.1
7906   sterlet           Acipenser ruthenus               GCA_010645085.1
7906   sterlet           Acipenser ruthenus               GCA_902713425.2
7906   sterlet           Acipenser ruthenus               GCA_902713435.1
7906   sterlet           Acipenser ruthenus               GCA_902713435.2
7904   white sturgeon    Acipenser transmontanus          GCA_020113195.1


結果、（2022年時点では）チョウザメゲノムは Sterletばかりであること、`GCF_` から始まる RefSeq のデータは2件しかないことが分かりました。

ではアセンブリレベルが complete のものはどれほどあるのでしょう。

In [2]:
 datasets summary genome taxon 'acipenser' --assembly-level complete

{"total_count": 0}


…皆無でした。

### `jq` での JSONデータの抽出・加工

`jq` を学ぶには自分で動かして試せる Sandbox.Bio の [JSON wrangling with jq](https://sandbox.bio/tutorials?id=jq-intro) が便利です。
